<a href="https://colab.research.google.com/github/vvikasreddy/lexically_constrained_beam_search_/blob/main/constraints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
# importing necessary libraries

from datasets import load_dataset
from collections import defaultdict
from tqdm import tqdm

In [22]:
# loading the wmt dataset, turkish to english

ds = load_dataset("wmt/wmt16", "tr-en")

In [4]:
def get_ngrams(src, n = 2, ):
  """
  The function returns the n_grams of length n, from the given sentence."""

  src = src.split(" ")
  src = [tuple(src[i:i+n]) for i in range(len(src) - n + 1)]
  return src

In [59]:
def constraints():

  """
  The function returns the """

  src_ngrams_count = defaultdict(int)
  tgt_ngrams_count = defaultdict(int)
  dict_pairs = defaultdict(int)

  dict_pairs_count = 0
  total_src_words = 0
  total_tgt_words = 0


  count = 0
  for n_gram in range(3, 4):

    for doc in tqdm(ds["train"]["translation"]):

      # get the src, and tgt
      src = doc["tr"]
      tgt = doc["en"]

      # get the ngrams, default n = 2
      src_ngrams = get_ngrams(src, n = n_gram)
      tgt_ngrams = get_ngrams(tgt, n = n_gram)


      # count the occurences of each of the ngram
      for tgt_ngram in tgt_ngrams:
        total_tgt_words += n_gram
        tgt_ngrams_count[tgt_ngram] += 1

      for src_ngram in src_ngrams:
        total_src_words += n_gram
        src_ngrams_count[src_ngram] += 1

      # # count the combined co-occurence of the ngram

      for src_ngram in src_ngrams:
        for tgt_ngram in tgt_ngrams:
          dict_pairs_count+= min(src_ngrams_count[src_ngram], tgt_ngrams_count[tgt_ngram])
          dict_pairs[(src_ngram, tgt_ngram)] += min(src_ngrams_count[src_ngram], tgt_ngrams_count[tgt_ngram])
      # if count %1000 == 0:
      #   print(count)
      # count += 1

  return dict_pairs_count, dict_pairs, src_ngrams_count, tgt_ngrams_count, total_src_words, total_tgt_words



In [60]:
dict_pairs_count, dicr_pairs, src_ngrams_count, tgt_ngrams_count, total_src_words, total_tgt_words =  constraints()

100%|██████████| 205756/205756 [03:14<00:00, 1055.27it/s]


In [54]:
print(len(src_ngrams_count))
print(max(src_ngrams_count.values()))
key, value = max(src_ngrams_count.items(), key=lambda x: x[1])
print(key, value)
for i in src_ngrams_count.items():
  print(i)
  break

2610798
4677
('Southeast', 'European', 'Times') 4677
(("Kosova'nın", 'özelleştirme', 'süreci'), 1)


In [64]:
# min_count = 3
# filtered_src_tgt = {k: v for k, v in src_ngrams_count.items() if v >= min_count}
# filtered_tgt = {k: v for k, v in tgt_ngrams_count.items() if v >= min_count}

print(len(dicr_pairs))
min_count = 3
filtered_src_tgt = {k: v for k, v in dicr_pairs.items() if v >= min_count}


78274550


In [63]:
len(filtered_src_tgt)

3404578

In [65]:
for i in filtered_src_tgt.items():
  print(i)
  break

((('Southeast', 'European', 'Times'), ('By', 'Muhamet', 'Brajshori')), 8775)


In [34]:
def get_count_dict_pairs(filtered_src, filtered_tgt, src_ngrams_count, tgt_ngrams_count):

  dict_pairs = defaultdict(int)

  count = 0
  for doc in tqdm(ds["train"]["translation"]):

    # get the src, and tgt
    src = doc["tr"]
    tgt = doc["en"]

    # get the ngrams, default n = 2
    src_ngrams = get_ngrams(src)
    tgt_ngrams = get_ngrams(tgt)

    # # count the combined co-occurence of the ngram

    for src_ngram in src_ngrams:
      if src_ngram not in filtered_src: continue
      for tgt_ngram in tgt_ngrams:

        if tgt_ngram not in filtered_tgt: continue
        dict_pairs[(src_ngram, tgt_ngram)] += min(src_ngrams_count[src_ngram], tgt_ngrams_count[tgt_ngram])

    # if count %1000 == 0:
    #   print(count)
    count += 1

  return dict_pairs
# dict_pairs =  get_count_dict_pairs()

In [35]:
# len(dict_pairs)
# import gc

In [36]:
# dict_pairs

In [37]:
# cleaning the unnecessary details



In [38]:
import math

def calculate_pmi(dict_pairs_count, dict_pairs, src_ngrams_count, tgt_ngrams_count, total_tgt_words, total_src_words):
    ignored_count = 0
    pmi_dict = {}
    count = 0

    total_dict_pairs = dict_pairs_count

    for (src_ngram, tgt_ngram), count in tqdm(dict_pairs.items()):
        p_src_tgt = dict_pairs[(src_ngram, tgt_ngram)] / total_dict_pairs

        p_src = src_ngrams_count[src_ngram] / total_src_words
        p_tgt = tgt_ngrams_count[tgt_ngram] / total_tgt_words

        if p_src == 0 or p_tgt == 0:
          pmi = 0
        else:
          pmi = math.log2(p_src_tgt / (p_src * p_tgt))/ -math.log2(p_src_tgt)


        if pmi > 0.9:
          pmi_dict[(src_ngram, tgt_ngram)] = pmi
        else:
          ignored_count += 1

    return pmi_dict


In [39]:
def get_constraints():

  # get the count and filtering if occurences is less than min_count, set 5
  dict_pairs_count, src_ngrams_count, tgt_ngrams_count, total_src_words, total_tgt_words =  constraints()
  min_count = 3
  filtered_src = {k: v for k, v in src_ngrams_count.items() if v >= min_count}
  filtered_tgt = {k: v for k, v in tgt_ngrams_count.items() if v >= min_count}

  dict_pairs =  get_count_dict_pairs(src_ngrams_count=src_ngrams_count, tgt_ngrams_count= tgt_ngrams_count, filtered_src=filtered_src, filtered_tgt=filtered_tgt)


  pmi_scores = calculate_pmi(dict_pairs_count, dict_pairs, filtered_src, filtered_tgt, total_tgt_words, total_src_words)


  # dictionary to only return the pair with max PMI value; since, there might be a different tgt value for the same src.
  max_pmi_dict = {}


  for (src, tgt), pmi_value in pmi_scores.items():
      if src not in max_pmi_dict:
          max_pmi_dict[src] = (tgt, pmi_value)
      elif pmi_value > max_pmi_dict[src][1]:
        max_pmi_dict[src] = (tgt, pmi_value)

  return max_pmi_dict

In [40]:
pmi_scores = get_constraints()

  0%|          | 0/3 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [17]:
len(pmi_scores)

4252

In [18]:
max(pmi_scores.values())

(('you', 'satisfied'), 0.9065577719262932)

In [30]:
x = pmi_scores.keys()
y = 0
for i in x:
  y = max(len(i), y)
print(y)

2


In [26]:
x

dict_keys([('Southeast', 'European'), ('European', 'Times'), ('Times', 'için'), ('için', "Priştine'den"), ("Priştine'den", 'Muhamet'), ('Muhamet', "Brayşori'nin"), ("Brayşori'nin", 'haberi'), ('haberi', '--'), ('beş', 'yıl'), ('yıl', 'önce'), ('uzun', 'vadeli'), ('vadeli', 'bir'), ('Sırp', 'makamları'), ("SETimes'a", 'konuşan'), ('2006', 'yılında'), ('Miloseviç', 'rejiminin'), ('ve', 'bu'), ('son', 'yıllarda'), ('bir', 'veya'), ('Balkan', 'Politika'), ('ve', 'yolsuzluk'), ('şu', 'anda'), ('bir', 'anlaşma'), ('siyasi', 'irade'), ('çok', 'sayıda'), ('Meclis', 'Başkanı'), ('Başkanı', 'Yakup'), ('Yakup', 'Krasniki'), ('2007', 'yılında'), ('bir', 'saldırı'), ('için', "Üsküp'ten"), ("Lutovska'nın", 'haberi'), ('13', 'yıl'), ('yıl', 'hapse'), ('hapse', 'mahkum'), ('[Tomislav', 'Georgiev/SETimes]'), ('ve', 'başka'), ('sahibi', 'olan'), ('ve', 'vergi'), ('ve', 'dört'), ('karşı', 'yükümlülüklerini'), ('yükümlülüklerini', 'yerine'), ("SETimes'a", 'verdiği'), ('verdiği', 'demeçte,'), ('olmak', 'üz

In [21]:
pmi_scores

{('Southeast', 'European'): (('Southeast', 'European'), 3.510950766389547),
 ('European', 'Times'): (('European', 'Times'), 3.3402701099081127),
 ('Times', 'için'): (('for', 'Southeast'), 3.2944638423783204),
 ('için', "Priştine'den"): (('in', 'Pristina'), 1.5861641326208846),
 ("Priştine'den", 'Muhamet'): (('By', 'Muhamet'), 1.211913096938427),
 ('Muhamet', "Brayşori'nin"): (('Brajshori', 'for'), 0.9445028767045213),
 ("Brayşori'nin", 'haberi'): (('Brajshori', 'for'), 0.949787519675183),
 ('haberi', '--'): (('for', 'Southeast'), 2.3478726164445782),
 ('beş', 'yıl'): (('five', 'years'), 1.1513733214657598),
 ('yıl', 'önce'): (('years', 'ago,'), 1.200562334773239),
 ('uzun', 'vadeli'): (('a', 'long-term'), 0.9654738919701555),
 ('vadeli', 'bir'): (('a', 'long-term'), 1.0456761177656284),
 ('Sırp', 'makamları'): (('Serbian', 'authorities'), 0.9343864196846737),
 ("SETimes'a", 'konuşan'): (('told', 'SETimes.'), 1.7176790245526945),
 ('2006', 'yılında'): (('in', '2006.'), 1.078804893481608